# Tree Clustering Code 

* Code is based on this implementation of decision tree: https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/
* This code builds decision tree using tree distances 
* Data Requirements:
    - numerical 
    - the first column must be the label

In [1]:
# sort values first - binary search 
# distance matrix - rather than for every pair (all-to-all shortest pathes )
# try hashing data structures 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import scipy.stats as sc
#import shap
#import lime
import sklearn 
import warnings
#import xgboost

import multiprocessing as mp
import time
import pickle 
#import interpret
import collections
import math
import random
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import  RandomForestRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn import preprocessing
#from interpret.glassbox import ExplainableBoostingClassifier
#from interpret import show
from sklearn.preprocessing import LabelEncoder
#from interpret.glassbox import ExplainableBoostingRegressor
#from interpret import show
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import AdaBoostRegressor
%matplotlib inline
import os, sys
#import statsmodels.api as sm
sys.path.append(os.path.abspath("../../../"))

pd.set_option('display.max_columns', None)
#import eli5
#from eli5.sklearn import PermutationImportance


In [2]:
train_final_dist = pd.read_csv("train_final_dist.csv")
test_final_dist = pd.read_csv("test_final_dist.csv")
full_data_train = pd.read_csv("full_data_train.csv")
full_data_test = pd.read_csv("full_data_test.csv")

In [3]:
train_final_dist = train_final_dist[['i1', 'i2', 'tree_dist']]
test_final_dist = test_final_dist[["i1", "i2", "tree_dist"]]

In [ ]:
df_train = full_data_train[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Deck"]]
df_test = full_data_test[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Deck"]]

In [ ]:
dataset_train = df_train.to_numpy()
dataset_test = df_test.to_numpy()

In [ ]:
small_df_train = full_data_train_no_clust.head(50)
small_df_test = full_data_test_no_clust.head(50)

In [ ]:
small_dataset_train = small_df_test.to_numpy()
small_dataset_test = small_df_train.to_numpy()

In [53]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset, df):
    feature = df.columns[index]
    left, right = list(), list()
    indleft, indright = list(), list()
    left_df, right_df = pd.DataFrame(columns = df.columns), pd.DataFrame(columns = df.columns)
    for i, row in df.iterrows():
        if row[feature] < value:
            indleft.append(i)
            left_df = left_df.append(row) 
        else:
            indright.append(i)
            right_df = right_df.append(row) 
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right, indleft, indright, left_df, right_df
 
# Calculate the distance index for a split dataset
def distance_index(left, right, indleft, indright):
    mean_left_dist = 0
    mean_right_dist = 0
    for i in range(len(indleft)):
        for j in range(i+1, len(indleft)):
            nodei_df = train_final_dist[train_final_dist['i1'] == indleft[i]]
            nodei_j_df = nodei_df[nodei_df['i2'] == indleft[j]]
            dist =  float(nodei_j_df["tree_dist"])
            mean_left_dist += dist 
    for i in range(len(indright)):
        for j in range(i+1, len(indright)):
            nodei_df = test_final_dist[test_final_dist['i1'] == indright[i]]
            nodei_j_df = nodei_df[nodei_df['i2'] == indright[j]]
            dist =  float(nodei_j_df["tree_dist"])
            mean_right_dist += dist
    mean_left_dist = 0
    mean_right_dist = 0
    if len(indleft) > 0:
        mean_left_dist = mean_left_dist / (len(indleft) + 1)
    if len(indright) > 0:
        mean_right_dist = mean_right_dist / (len(indright) +1)
    out = 0
    if 
    return mean_left_dist * len(left) / (len(left) + len(right)) + mean_right_dist * len(right) / (len(left) + len(right))

# Select the best split point for a dataset
def get_split(dataset, df):
    class_values = list(set(row[0] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(1, len(dataset[0])):
        for row in dataset:
            left, right, indleft, indright, left_df, right_df = test_split(index, row[index], dataset, df)
            groups = left, right, left_df, right_df
            dist = distance_index(left, right, indleft, indright)
            if dist < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], dist, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [54]:
# Create a leaf node value
def to_terminal(group):
    outcomes = [row[0] for row in group]
    return max(set(outcomes), key=outcomes.count)
 
# Create child splits for a node or make leaf
def split(node, max_depth, min_size, depth):
    left, right, left_df, right_df = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        #print('here')
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, left_df)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        #print('here')
        node['right'] = to_terminal(right)
    else:
        #print('here')
        node['right'] = get_split(right, right_df)
        split(node['right'], max_depth, min_size, depth+1)

In [55]:
# Build a decision tree
def build_tree(train, df, max_depth, min_size):
    root = get_split(train, df)
    split(root, max_depth, min_size, 1)
    return root
 
# Print a decision tree
def print_tree(node, df, depth=0):
    if isinstance(node, dict):
        print('%s[%s < %.3f]' % ((depth*' ', (df.columns[node['index']]), node['value'])))
        print_tree(node['left'], df, depth+1)
        print_tree(node['right'], df, depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [56]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [69]:
X_train = small_df_train[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Deck"]]
y_train = small_df_train['Survived']
X_test = small_df_test[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Deck"]]
y_test = small_df_test['Survived']

In [70]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [71]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

[[35  1]
 [ 6  8]]


In [62]:
tree_small = build_tree(small_dataset_train, small_df_train, 5, 20)

In [63]:
print_tree(tree_small, small_df_train)

[Fare < 9.000]
 [Age < 27.000]
  [0]
  [0]
 [Age < 29.000]
  [0]
  [0]


In [72]:
y_pred_dt = []
for row in small_dataset_test:
    y_pred_dt.append(predict(tree_small, row))

In [73]:
len(y_test)

50

In [74]:
conf_mat = confusion_matrix(y_test, y_pred_dt)
print(conf_mat)

[[36  0]
 [14  0]]


In [75]:
# weighted precision, recall, f-score, support 
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(y_test, y_pred, average ='weighted'))

(0.8635230352303523, 0.86, 0.8493280632411068, None)


In [76]:
print(precision_recall_fscore_support(y_test, y_pred_dt, average ='weighted'))

(0.5184, 0.72, 0.6027906976744185, None)


/Users/aparnacalambur/anaconda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# create distances based on test distances 

# applying the decision rule to test set (supervised)
# applying to train (semi-supervised notion of clustering)
# - we are using distance generated by test set and clustering based on that 
# - we use distance metric (metric = forest) to get distance matrix for test set - training set provides metric
# - you can also build distance from both trained and test ?????
# - when test bigger than training - semi-supervised (more unlabeled than labeled)
# we are doing train/test split - but how we predict we use majority vote of training within cluster
# build from: training + test set OR just train

# how to evaluate clusters in the end: 

In [ ]:
train_final_dist

In [44]:
nodei_df = train_final_dist[train_final_dist['i1'] == 283]
nodei_j_df = nodei_df[nodei_df['i2'] == 284]
float(nodei_j_df["tree_dist"])

14.782178217821784

,i1,i2,tree_dist
0,0,1,10.752475
1,0,2,10.336634
2,0,3,10.970297
3,0,4,5.396040
4,0,5,9.287129
...,...,...,...
193748,619,621,13.227723
193749,619,622,10.168317
193750,620,621,11.425743
193751,620,622,12.980198


In [88]:
def get_node_depths(tree1):
    def get_node_depths_(current_node, current_depth, l, r, depths):
        depths += [current_depth]
        if l[current_node] != -1 and r[current_node] != -1:
            get_node_depths_(l[current_node], current_depth + 1, l, r, depths)
            get_node_depths_(r[current_node], current_depth + 1, l, r, depths)

    depths = []
    get_node_depths_(0, 0, tree1.tree_.children_left, tree1.tree_.children_right, depths) 
    return np.array(depths)

def get_shared_nodes(i1,i2,node_indicator,n_nodes):
    sample_ids = [i1, i2]
    common_nodes = (node_indicator.toarray()[sample_ids].sum(axis=0) ==
                    len(sample_ids))
    
    common_node_id = np.arange(n_nodes)[common_nodes]
    
    return common_node_id

# Tree distance between nodes n1, n2 = depth(n1) + depth(n2) - 2 depth(LCA)
def distance_between_samples(indexes,depths,leaves,node_indicator,n_nodes):
    i1 = indexes[0]
    i2 = indexes[1]
    leaf_node1 = leaves[i1]
    leaf_node2 = leaves[i2]
    depth_node1 = depths[leaf_node1]
    depth_node2 = depths[leaf_node2]
    ancestors = get_shared_nodes(i1,i2,node_indicator,n_nodes)
    depth_LCA = max(depths[ancestors])
    
    dist = depth_node1 + depth_node2 - 2*depth_LCA
    if dist < 0:
        #print(len(depths))
        depths[ancestors]
    return dist

In [92]:
# bootstraps data and builds a tree, then calculates pairwise distances on the data instances relative to the tree
# Tree distance is calculated via lowest common ancestor
def build_tree(xTrain,yTrain,xTest):
    train = xTrain.copy()
    train['y'] = yTrain
    train1 = train.sample(n = len(train), replace = True) 
    yTrain1 = train1['y']
    xTrain1 = train1.drop('y',axis = 1)
    gc.collect()
    estimator = DecisionTreeClassifier().fit(xTrain1,yTrain1)
    n_nodes = estimator.tree_.node_count
    depths = get_node_depths(estimator)
    leaves_train = estimator.apply(xTrain)
    leaves_test = estimator.apply(xTest)
    node_indicator_test = estimator.decision_path(xTest)
    node_indicator_train = estimator.decision_path(xTrain)
    train_comb = list(itertools.combinations(range(0,len(xTrain)), 2))
    test_comb = list(itertools.combinations(range(0,len(xTest)), 2))
    print(type(leaves_test))
    ### Train Distances
    train_distances = []
    for indexes in train_comb:
        dist = distance_between_samples(indexes,depths,leaves_train,node_indicator_train,n_nodes)
        train_distances.append([indexes[0],indexes[1],dist])

    ### Test Distances
    test_distances = []
    for indexes in test_comb:
        dist = distance_between_samples(indexes,depths,leaves_train,node_indicator_test,n_nodes)
        test_distances.append([indexes[0],indexes[1],dist])

    train_dist_df = pd.DataFrame(train_distances, columns = ['i1','i2','tree_dist'])
    test_dist_df = pd.DataFrame(test_distances, columns = ['i1','i2','tree_dist'])
    return(estimator,train_dist_df,test_dist_df)

In [90]:
import itertools
import gc

In [91]:
estimator,train_dist_df,test_dist_df = build_tree(X_train,y_train,X_test)

<class 'list'>


In [108]:
def get_node_count(node, df, depth=0):
    num_nodes = 0
    if isinstance(node, dict):
        num_nodes += 1 + get_node_count(node['left'], df, depth+1)
        num_nodes += 1 + get_node_count(node['right'], df, depth+1)
    else:
        num_nodes += 1
    return num_nodes

In [110]:
tree_small

{'index': 6,
 'left': {'index': 3, 'left': 0, 'right': 0, 'value': 27},
 'right': {'index': 3, 'left': 0, 'right': 0, 'value': 29},
 'value': 9}

In [ ]:

n_nodes = estimator.tree_.node_count
depths = get_node_depths(estimator)
leaves_train = estimator.apply(xTrain)
leaves_test = estimator.apply(xTest)
node_indicator_test = estimator.decision_path(xTest)
node_indicator_train = estimator.decision_path(xTrain)
train_comb = list(itertools.combinations(range(0,len(X_train)), 2))
test_comb = list(itertools.combinations(range(0,len(X_test)), 2))

### Test Distances
test_distances = []
for indexes in test_comb:
    dist = distance_between_samples(indexes,depths,leaves_train,node_indicator_test,n_nodes)
    test_distances.append([indexes[0],indexes[1],dist])

train_dist_df = pd.DataFrame(train_distances, columns = ['i1','i2','tree_dist'])
test_dist_df = pd.DataFrame(test_distances, columns = ['i1','i2','tree_dist'])
return([estimator,train_dist_df,test_dist_df])